In [1]:
import awkward as ak
from coffea import nanoevents
from coffea.nanoevents.methods.base import NanoEventsArray
from coffea.nanoevents.methods.nanoaod import FatJetArray, GenParticleArray
from typing import Union
import numpy as np

In [2]:
d_PDGID = 1
u_PDGID = 2
s_PDGID = 3
c_PDGID = 4
b_PDGID = 5
g_PDGID = 21
TOP_PDGID = 6

ELE_PDGID = 11
vELE_PDGID = 12
MU_PDGID = 13
vMU_PDGID = 14
TAU_PDGID = 15
vTAU_PDGID = 16

Z_PDGID = 23
W_PDGID = 24
HIGGS_PDGID = 25
Y_PDGID = 35

b_PDGIDS = [511, 521, 523]

GRAV_PDGID = 39

GEN_FLAGS = ["fromHardProcess", "isLastCopy"]

def to_label(array: ak.Array) -> ak.Array:
    return ak.values_astype(array, np.int32)

In [3]:
events = nanoevents.NanoEventsFactory.from_root(
    "nano_mc2017_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1950 branches) at 0x000103c8c670>, taking first instance
  warnings.warn(
/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1950 branches) at 0x000103c8c670>, taking first instance
  warnings.warn(
/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1950 branches) at 0x000103c8c670>, taking first instance
  warnings.warn(
/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1950 branches) at 0x000103c8c6

In [4]:
events

<NanoEventsArray [<event 1:5575:5574008>, ... ] type='46000 * event'>

In [5]:
def get_pid_mask(
    genparts: GenParticleArray,
    pdgids: Union[int, list],
    ax: int = 2,
    byall: bool = True,
) -> ak.Array:
    """                                                                                                                                                                                                               
    Get selection mask for gen particles matching any of the pdgIds in ``pdgids``.                                                                                                                                    
    If ``byall``, checks all particles along axis ``ax`` match.                                                                                                                                                       
    """
    gen_pdgids = abs(genparts.pdgId)

    if type(pdgids) == list:
        mask = gen_pdgids == pdgids[0]
        for pdgid in pdgids[1:]:
            mask = mask | (gen_pdgids == pdgid)
    else:
        mask = gen_pdgids == pdgids

    return ak.all(mask, axis=ax) if byall else mask

In [6]:
JET_DR = 0.8
fatjet = ak.firsts(events.FatJet)
tops = events.GenPart[get_pid_mask(events.GenPart, TOP_PDGID, byall=False) * events.GenPart.hasFlags(GEN_FLAGS)]
matched_tops = tops[fatjet.delta_r(tops) < JET_DR]
matched_tops_mask = ak.any(fatjet.delta_r(tops) < JET_DR, axis=1)
num_matched_tops = ak.sum(fatjet.delta_r(matched_tops) < JET_DR, axis=1)

# all daughters!
daughters = ak.flatten(tops.distinctChildren, axis=2)
daughters = daughters[daughters.hasFlags(GEN_FLAGS)]
daughters_pdgId = abs(daughters.pdgId)

matched_topdaus_mask = ak.any(fatjet.delta_r(daughters) < JET_DR, axis=1)
    
wboson_daughters = ak.flatten(daughters[(daughters_pdgId == 24)].distinctChildren, axis=2)
wboson_daughters = wboson_daughters[wboson_daughters.hasFlags(GEN_FLAGS)]
wboson_daughters_pdgId = abs(wboson_daughters.pdgId)

bquark = daughters[(daughters_pdgId == 5)]
neutrinos = (
    (wboson_daughters_pdgId == vELE_PDGID)
    | (wboson_daughters_pdgId == vMU_PDGID)
    | (wboson_daughters_pdgId == vTAU_PDGID)
)
leptons = (
    (wboson_daughters_pdgId == ELE_PDGID)
    | (wboson_daughters_pdgId == MU_PDGID)
    | (wboson_daughters_pdgId == TAU_PDGID)
)
quarks = ~leptons & ~neutrinos

cquarks = (
    (wboson_daughters_pdgId == c_PDGID)
)
electrons = (
    (wboson_daughters_pdgId == ELE_PDGID)
)
muons = (
    (wboson_daughters_pdgId == ELE_PDGID)
)
taus = (
    (wboson_daughters_pdgId == TAU_PDGID)
)

# look at tau decay
taudaughters = wboson_daughters[taus].children
taudaughters = taudaughters[taudaughters.hasFlags(["isLastCopy"])]
taudaughters_pdgId = abs(taudaughters.pdgId)
taudecay = (
    # pions/kaons (hadronic tau) * 1                                                                                                                                                                              
    (
        ak.sum(
        (taudaughters_pdgId == ELE_PDGID) | (taudaughters_pdgId == MU_PDGID),
        axis=2,
        )
        == 0
    )
    * 1
    # 1 electron * 3                                                                                                                                                                                              
    + (ak.sum(taudaughters_pdgId == ELE_PDGID, axis=2) == 1) * 3
    # 1 muon * 5                                                                                                                                                                                                  
    + (ak.sum(taudaughters_pdgId == MU_PDGID, axis=2) == 1) * 5
)
# flatten taudecay - so painful                                                                                                                                                                                   
taudecay = ak.sum(taudecay, axis=-1)

# number of matched daughters
num_m_quarks_nob = ak.sum(fatjet.delta_r(wboson_daughters[quarks]) < JET_DR, axis=1)
num_m_bquarks = ak.sum(fatjet.delta_r(bquark) < JET_DR, axis=1)
num_m_cquarks = ak.sum(fatjet.delta_r(wboson_daughters[cquarks]) < JET_DR, axis=1)
num_m_leptons = ak.sum(fatjet.delta_r(wboson_daughters[leptons]) < JET_DR, axis=1)
num_m_electrons = ak.sum(fatjet.delta_r(wboson_daughters[electrons]) < JET_DR, axis=1)
num_m_muons = ak.sum(fatjet.delta_r(wboson_daughters[muons]) < JET_DR, axis=1)
num_m_taus = ak.sum(fatjet.delta_r(wboson_daughters[taus]) < JET_DR, axis=1)

matched_tops_mask = ak.any(fatjet.delta_r(tops) < JET_DR, axis=1)
matched_topdaus_mask = ak.any(fatjet.delta_r(daughters) < JET_DR, axis=1)
matched_mask = matched_tops_mask & matched_topdaus_mask
    
genVars = {
    "fj_Top_isMatched": matched_mask, # at least one top and one daugther matched..                                                                                                                                                                                         
    "fj_Top_numMatched": num_matched_tops, # number of tops matched                                                                                                                                                                                                       
    "fj_Top_nquarksnob": num_m_quarks_nob, # number of quarks from W decay (not b) matched in dR                                                                                                                                                                            
    "fj_Top_nbquarks": num_m_bquarks, # number of b quarks ..                                                                                                                                                                                                               
    "fj_Top_ncquarks": num_m_cquarks, # number of c quarks ..                                                                                                                                                                                                               
    "fj_Top_nleptons": num_m_leptons, # number of leptons ..                                                                                                                                                                                                                
    "fj_Top_nele": num_m_electrons, # number of electrons...                                                                                                                                                                                                                
    "fj_Top_nmu": num_m_muons, # number of muons...                                                                                                                                                                                                                         
    "fj_Top_ntau": num_m_taus,  # number of taus...                                                                                                                                                                                                                         
    "fj_Top_taudecay": taudecay, # taudecay (1: hadronic, 3: electron, 5: muon)                                                                                                                                                                                             
}
    
# define labels
label_Top_bWqq_0c = to_label((num_m_quarks_nob==2) & (num_m_bquarks==1) & (num_m_cquarks==0) & (num_m_leptons==0))
label_Top_bWqq_1c = to_label((num_m_quarks_nob==2) & (num_m_bquarks==1) & (num_m_cquarks==1) & (num_m_leptons==0))
label_Top_Wqq_0c = to_label((num_m_quarks_nob==2) & (num_m_bquarks==0) & (num_m_cquarks==0) & (num_m_leptons==0))
label_Top_Wqq_1c = to_label((num_m_quarks_nob==2) & (num_m_bquarks==0) & (num_m_cquarks==1) & (num_m_leptons==0))

label_Top_bWq_0c = to_label((num_m_quarks_nob==1) & (num_m_bquarks==1) & (num_m_cquarks==0) & (num_m_leptons==0))
label_Top_bWq_1c = to_label((num_m_quarks_nob==1) & (num_m_bquarks==1) & (num_m_cquarks==1) & (num_m_leptons==0))
label_Top_Wq_0c = to_label((num_m_quarks_nob==1) & (num_m_bquarks==0) & (num_m_cquarks==0) & (num_m_leptons==0))
label_Top_Wq_1c = to_label((num_m_quarks_nob==1) & (num_m_bquarks==0) & (num_m_cquarks==1) & (num_m_leptons==0))

label_Top_bWev = to_label((num_m_quarks_nob==0) & (num_m_bquarks==1) & (num_m_leptons==1) & (num_m_electrons==1))
label_Top_bWmv = to_label((num_m_quarks_nob==0) & (num_m_bquarks==1) & (num_m_leptons==1) & (num_m_muons==1))
label_Top_Wev = to_label((num_m_quarks_nob==0) & (num_m_bquarks==0) & (num_m_leptons==1) & (num_m_electrons==1))
label_Top_Wmv = to_label((num_m_quarks_nob==0) & (num_m_bquarks==0) & (num_m_leptons==1) & (num_m_muons==1))

label_Top_bWtauhv = to_label((num_m_quarks_nob==0) & (num_m_bquarks==1) & (num_m_leptons==1) & (num_m_taus==1) & (taudecay == 1))
label_Top_bWtauev = to_label((num_m_quarks_nob==0) & (num_m_bquarks==1) & (num_m_leptons==1) & (num_m_taus==1) & (taudecay == 3))
label_Top_bWtaumv = to_label((num_m_quarks_nob==0) & (num_m_bquarks==1) & (num_m_leptons==1) & (num_m_taus==1) & (taudecay == 5))
label_Top_Wtauhv = to_label((num_m_quarks_nob==0) & (num_m_bquarks==0) & (num_m_leptons==1) & (num_m_taus==1) & (taudecay == 1))
label_Top_Wtauev = to_label((num_m_quarks_nob==0) & (num_m_bquarks==0) & (num_m_leptons==1) & (num_m_taus==1) & (taudecay == 3))
label_Top_Wtaumv = to_label((num_m_quarks_nob==0) & (num_m_bquarks==0) & (num_m_leptons==1) & (num_m_taus==1) & (taudecay == 5))

label_Top_bsWqqWqs = to_label((num_m_quarks_nob>2) & (num_m_bquarks>1))
label_Top_WqqWq = to_label((num_m_quarks_nob>2) & (num_m_bquarks==0))

In [7]:
events = nanoevents.NanoEventsFactory.from_root(
    "nano_mc2017_22.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nBHadrons in <TTree 'Events' (1955 branches) at 0x00012a67b880>, taking first instance
  warnings.warn(
/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15SubJet_nCHadrons in <TTree 'Events' (1955 branches) at 0x00012a67b880>, taking first instance
  warnings.warn(
/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nBHadrons in <TTree 'Events' (1955 branches) at 0x00012a67b880>, taking first instance
  warnings.warn(
/Users/cmantill/miniconda3/envs/coffea/lib/python3.8/site-packages/coffea/nanoevents/mapping/uproot.py:86: UserWarning: Found duplicate branch FatJetAK15_nCHadrons in <TTree 'Events' (1955 branches) at 0x00012a67b8

In [8]:
events.FatJet

<FatJetArray [[], [FatJet], ... [FatJet, FatJet]] type='18000 * var * fatJet'>

In [9]:
fatjet = ak.firsts(events.FatJet)

dau_pdgid = W_PDGID
higgs = events.GenPart[get_pid_mask(events.GenPart, HIGGS_PDGID, byall=False) * events.GenPart.hasFlags(GEN_FLAGS)]
higgs_children = higgs.children
children_mask = get_pid_mask(higgs_children, [W_PDGID], byall=False)
higgs_children = higgs_children[children_mask]

signal_mask = ak.firsts(ak.all(abs(higgs.children.pdgId) == dau_pdgid, axis=2))

matched_higgs = higgs[ak.argmin(fatjet.delta_r(higgs), axis=1, keepdims=True)][:, 0]
matched_higgs_children = matched_higgs.children

children_mask = get_pid_mask(matched_higgs_children, [W_PDGID], byall=False)
matched_higgs_children = matched_higgs_children[children_mask]

children_mass = matched_higgs_children.mass
v_star = ak.firsts(matched_higgs_children[ak.argmin(children_mass, axis=1, keepdims=True)])
v = ak.firsts(matched_higgs_children[ak.argmax(children_mass, axis=1, keepdims=True)])
is_hww_matched = ak.any(children_mask, axis=1)

def replace_photon(daughters):
    # look at those decays where W W photon (24 24 22)
    # distinctChildren results in [] [22]
    # children results in (24 24 22)
    # children.children results in [13, 14], [3, 4] []
    try:
        flat = ak.flatten(abs(daughters.pdgId),axis=3)
    except:
        flat = abs(daughters.pdgId)
    try:
        flat = ak.flatten(flat,axis=2)
    except:
        flat = flat
    photon_mask = ak.any(flat == 22,axis=1)

    try:
        daus = ak.flatten(daughters.children,axis=4)
    except:
        #daus = daughters.children
        daus = ak.flatten(daughters.children,axis=2)
    
    # use a where condition to get all possible daughters
    all_daughters = ak.where(photon_mask, daus, daughters)
    
    if ak.any(photon_mask):
        all_daughters = replace_photon(all_daughters)
    else:
        return all_daughters
    
    if all_daughters is not None:
        return all_daughters
        
all_daus = replace_photon(higgs_children.children)
all_daus = ak.flatten(all_daus, axis=2)
all_daus_flat = ak.flatten(all_daus, axis=2)
all_daus_flat_pdgId = abs(all_daus_flat.pdgId)

# look at tau-daughters
taudaughters = all_daus[(all_daus_flat_pdgId == TAU_PDGID)].children
taudaughters = taudaughters[taudaughters.hasFlags(["isLastCopy"])]
taudaughters_pdgId = abs(taudaughters.pdgId)
taudaughters_pdgId = ak.flatten(taudaughters_pdgId,axis=2)
print(taudaughters_pdgId)
# 0: notaus
# 1: hadronictau 
# 3 and 4: electron?
# 5 and 6: muon?
# 9: muon and electron decay
taudecay = (
    # pions/kaons (hadronic tau) * 1                                                                                                                                                                          
    (
        ak.sum(ak.any((taudaughters_pdgId != ELE_PDGID) & (taudaughters_pdgId != MU_PDGID),axis=2),axis=1)
        >= 1
    )* 1
    # 1 electron * 3                                                                                                                                                                                          
    + (ak.sum(
            ak.any(taudaughters_pdgId == ELE_PDGID,axis=2),axis=1
        )
        >= 1) * 3
    # 1 muon * 5 
    + (ak.sum(
            ak.any(taudaughters_pdgId == MU_PDGID,axis=2),axis=1
        )
        >= 1) * 5
)

# the following tells you about the decay
num_quarks = ak.sum(all_daus_flat_pdgId <= b_PDGID, axis=1)
num_leptons = ak.sum((all_daus_flat_pdgId == ELE_PDGID) | (all_daus_flat_pdgId == MU_PDGID) | (all_daus_flat_pdgId == TAU_PDGID), axis=1)
num_electrons = ak.sum(all_daus_flat_pdgId == ELE_PDGID, axis=1)
num_muons = ak.sum(all_daus_flat_pdgId == MU_PDGID, axis=1)
num_taus = ak.sum(all_daus_flat_pdgId == TAU_PDGID, axis=1)
print(num_taus)

# the following tells you about the matching
# prongs except neutrino
neutrinos = (
    (all_daus_flat_pdgId == vELE_PDGID)
    | (all_daus_flat_pdgId == vMU_PDGID)
    | (all_daus_flat_pdgId == vTAU_PDGID)
)
leptons = (
    (all_daus_flat_pdgId == ELE_PDGID)
    | (all_daus_flat_pdgId == MU_PDGID)
    | (all_daus_flat_pdgId == TAU_PDGID)
)
# number of quarks excludes neutrino and leptons
num_m_quarks =  ak.sum(fatjet.delta_r(all_daus_flat[~neutrinos & ~leptons])  < JET_DR, axis=1)
num_m_leptons = ak.sum(fatjet.delta_r(all_daus_flat[leptons]) < JET_DR, axis=1)
num_m_cquarks = ak.sum(fatjet.delta_r(all_daus_flat[all_daus_flat.pdgId == b_PDGID]) < JET_DR, axis=1)

lep_daughters = all_daus_flat[leptons]
parent = ak.firsts(lep_daughters[fatjet.delta_r(lep_daughters) < JET_DR].distinctParent)
parent_mass = parent.mass
iswlepton = parent.mass == v.mass
iswstarlepton = parent.mass == v_star.mass

genHVVVars = {
    "fj_nquarks": num_m_quarks,
    "fj_ncquarks": num_m_cquarks,
    "fj_lepinprongs": num_m_leptons,
    "fj_H_VV_4q": to_label((num_quarks == 4) & (num_leptons==0)),
    "fj_H_VV_elenuqq": to_label((num_electrons == 1) & (num_quarks == 2) & (num_leptons==1)),
    "fj_H_VV_munuqq": to_label((num_muons == 1) & (num_quarks == 2) & (num_leptons==1)),
    "fj_H_VV_leptauelvqq": to_label((num_taus == 1) & ((taudecay == 3) | (taudecay == 4)) & (num_quarks == 2) & (num_leptons==1)),
    "fj_H_VV_leptaumuvqq": to_label((num_taus == 1) & ((taudecay == 5) | (taudecay == 6)) & (num_quarks == 2) & (num_leptons==1)),
    "fj_H_VV_hadtauvqq": to_label((num_taus == 1) & (taudecay == 1) & (num_quarks == 2) & (num_leptons==1)),
    "fj_H_VV_isVlepton": iswlepton,
    "fj_H_VV_isVstarlepton": iswstarlepton,
    "fj_H_VV_isMatched": is_hww_matched,
}

label_H_WqqWqq_0c = to_label( (num_quarks == 4) & (num_leptons==0) & (num_m_cquarks==0) & (num_m_quarks==4))
label_H_WqqWqq_1c = to_label( (num_quarks == 4) & (num_leptons==0) & (num_m_cquarks==1) & (num_m_quarks==4))
label_H_WqqWqq_2c = to_label( (num_quarks == 4) & (num_leptons==0) & (num_m_cquarks==2) & (num_m_quarks==4))
label_H_WqqWq_0c = to_label( (num_quarks == 4) & (num_leptons==0) & (num_m_cquarks==0) & (num_m_quarks==3))
label_H_WqqWq_1c = to_label( (num_quarks == 4) & (num_leptons==0) & (num_m_cquarks==1) & (num_m_quarks==3))
label_H_WqqWq_2c = to_label( (num_quarks == 4) & (num_leptons==0) & (num_m_cquarks==2) & (num_m_quarks==3))

label_H_WqqWev_0c = to_label( (num_electrons == 1) & (num_leptons==1) & (num_quarks == 2) & (num_m_leptons==1) & (num_m_quarks==2) & (num_m_cquarks==0) )
label_H_WqqWev_1c = to_label( (num_electrons == 1) & (num_leptons==1) & (num_quarks == 2) & (num_m_leptons==1) & (num_m_quarks==2) & (num_m_cquarks==1) )
label_H_WqqWmv_0c = to_label( (num_muons == 1) & (num_leptons==1) & (num_quarks == 2) & (num_m_leptons==1) & (num_m_quarks==2) & (num_m_cquarks==0) )
label_H_WqqWmv_1c = to_label( (num_muons == 1) & (num_leptons==1) & (num_quarks == 2) & (num_m_leptons==1) & (num_m_quarks==2) & (num_m_cquarks==1) )



[[[16, 111, 211], []], [[16, 111, 211], [], ... [], [], [], [], [[16, 111, 211], []]]
[1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, ... 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]


In [10]:
ak.sum(ak.any((taudaughters_pdgId != ELE_PDGID) & (taudaughters_pdgId != MU_PDGID),axis=2),axis=1)[6]

2

In [11]:
ak.sum(ak.any(taudaughters_pdgId == ELE_PDGID,axis=2),axis=1)[36]

1

In [12]:
taudecay[36]

4

In [13]:
morech = taudaughters.children.pdgId
for i,t in enumerate(taudaughters_pdgId):
    #print(i,t,morech[i],taudecay[i])
    if taudecay[i]==3:
        print(i,t,morech[i],taudecay[i])

11027 [[], [11, 11]] [[[], [[], []]]] 3


In [14]:
higgs = events.GenPart[get_pid_mask(events.GenPart, HIGGS_PDGID, byall=False) * events.GenPart.hasFlags(GEN_FLAGS)]
higgs_children = higgs.children

def replace_photon(daughters):
    # look at those decays where W W photon (24 24 22)
    # distinctChildren results in [] [22]
    # children results in (24 24 22)
    # children.children results in [13, 14], [3, 4] []
    try:
        flat = ak.flatten(abs(daughters.pdgId),axis=3)
    except:
        flat = abs(daughters.pdgId)
    try:
        flat = ak.flatten(flat,axis=2)
    except:
        flat = flat
    photon_mask = ak.any(flat == 22,axis=1)

    try:
        daus = ak.flatten(daughters.children,axis=4)
    except:
        #daus = daughters.children
        daus = ak.flatten(daughters.children,axis=2)
    
    # use a where condition to get all possible daughters
    all_daughters = ak.where(photon_mask, daus, daughters)
    
    if ak.any(photon_mask):
        all_daughters = replace_photon(all_daughters)
    else:
        return all_daughters
    
    if all_daughters is not None:
        return all_daughters
        
all_daus = replace_photon(higgs_children.children)
all_daus = ak.flatten(all_daus, axis=2)
all_daus_flat = ak.flatten(all_daus, axis=2)

In [15]:
np.set_printoptions(threshold=np.inf)
#print(taudecay.to_numpy())
print(np.unique(taudecay))

[0, 1, 3, 4, 6, 9]


In [16]:
all_daus_flat.pdgId

<Array [[-1, 2, 15, -16], ... [-15, 16, 1, -2]] type='18000 * var * ?int32[param...'>

In [17]:
(all_daus_flat.pdgId <= b_PDGID)[1]

<Array [True, False, False, True] type='4 * ?bool'>

In [18]:
for i,d in enumerate(all_daus_flat.pdgId):
    print(i,d,num_quarks[i],num_leptons[1])

0 [-1, 2, 15, -16] 2 2
1 [-15, 16, 13, -14] 0 2
2 [-1, 2, 1, -2] 4 2
3 [-13, 14, 3, -4] 2 2
4 [-3, 4, 3, -4] 4 2
5 [-15, 16, 11, -12] 0 2
6 [-15, 16, 1, -2] 2 2
7 [-1, 2, 1, -2] 4 2
8 [-1, 2, 3, -4] 4 2
9 [-1, 2, 1, -2] 4 2
10 [-3, 4, 3, -4] 4 2
11 [-3, 2, 11, -12] 2 2
12 [-13, 14, 3, -4] 2 2
13 [-3, 4, 1, -4] 4 2
14 [-3, 4, 3, -4] 4 2
15 [-11, 12, 3, -4] 2 2
16 [-11, 12, 3, -4] 2 2
17 [-3, 4, 11, -12] 2 2
18 [-1, 2, 1, -2] 4 2
19 [-1, 2, 1, -2] 4 2
20 [-1, 2, 15, -16] 2 2
21 [-1, 2, 3, -4] 4 2
22 [-1, 2, 1, -2] 4 2
23 [-11, 12, 1, -2] 2 2
24 [-3, 4, 1, -2] 4 2
25 [-1, 2, 11, -12] 2 2
26 [-3, 4, 3, -4] 4 2
27 [-3, 4, 3, -4] 4 2
28 [-13, 14, 13, -14] 0 2
29 [-3, 2, 13, -14] 2 2
30 [-1, 2, 11, -12] 2 2
31 [-1, 2, 13, -14] 2 2
32 [-1, 2, 11, -12] 2 2
33 [-3, 4, 3, -4] 4 2
34 [-1, 2, 13, -14] 2 2
35 [-3, 4, 3, -4] 4 2
36 [-3, 4, 15, -16] 2 2
37 [-11, 12, 13, -14] 0 2
38 [-3, 4, 1, -4] 4 2
39 [-1, 2, 3, -4] 4 2
40 [-3, 4, 1, -2] 4 2
41 [-1, 2, 13, -14] 2 2
42 [-13, 14, 1, -2] 2 2
43 [-15, 1

3539 [-13, 14, 3, -4] 2 2
3540 [-1, 2, 3, -4] 4 2
3541 [-5, 4, 15, -16] 2 2
3542 [-3, 4, 1, -2] 4 2
3543 [-3, 4, 1, -2] 4 2
3544 [-1, 2, 11, -12] 2 2
3545 [-13, 14, 1, -2] 2 2
3546 [-3, 4, 1, -2] 4 2
3547 [-3, 4, 15, -16] 2 2
3548 [-3, 4, 3, -4] 4 2
3549 [-1, 2, 3, -4] 4 2
3550 [-15, 16, 3, -4] 2 2
3551 [-3, 4, 15, -16] 2 2
3552 [-11, 12, 3, -4] 2 2
3553 [-1, 2, 1, -2] 4 2
3554 [-3, 4, 13, -14] 2 2
3555 [-3, 4, 11, -12] 2 2
3556 [-3, 4, 13, -14] 2 2
3557 [-15, 16, 1, -2] 2 2
3558 [-1, 2, 15, -16] 2 2
3559 [-1, 2, 1, -2] 4 2
3560 [-13, 14, 15, -16] 0 2
3561 [-3, 4, 15, -16] 2 2
3562 [-11, 12, 1, -2] 2 2
3563 [-3, 4, 1, -2] 4 2
3564 [-1, 2, 1, -4] 4 2
3565 [-1, 2, 15, -16] 2 2
3566 [-3, 4, 3, -4] 4 2
3567 [-3, 4, 1, -2] 4 2
3568 [-3, 4, 1, -2] 4 2
3569 [-1, 2, 3, -4] 4 2
3570 [-13, 14, 1, -2] 2 2
3571 [-1, 2, 1, -2] 4 2
3572 [-1, 2, 15, -16] 2 2
3573 [-1, 2, 1, -2] 4 2
3574 [-3, 4, 1, -2] 4 2
3575 [-1, 2, 3, -4] 4 2
3576 [-13, 14, 1, -2] 2 2
3577 [-3, 4, 11, -12] 2 2
3578 [-1, 2, 11, -12

5667 [-11, 12, 3, -4] 2 2
5668 [-3, 4, 1, -2] 4 2
5669 [-1, 2, 1, -4] 4 2
5670 [-3, 4, 3, -4] 4 2
5671 [-1, 2, 15, -16] 2 2
5672 [-11, 12, 1, -2] 2 2
5673 [-1, 2, 11, -12] 2 2
5674 [-3, 4, 1, -2] 4 2
5675 [-11, 12, 3, -4] 2 2
5676 [-3, 4, 1, -2] 4 2
5677 [-15, 16, 3, -4] 2 2
5678 [-1, 2, 1, -2] 4 2
5679 [-1, 2, 13, -14] 2 2
5680 [-1, 2, 1, -2] 4 2
5681 [-3, 4, 13, -14] 2 2
5682 [-1, 2, 1, -2] 4 2
5683 [-3, 4, 15, -16] 2 2
5684 [-13, 14, 1, -2] 2 2
5685 [-1, 2, 15, -16] 2 2
5686 [-1, 2, 3, -4] 4 2
5687 [-1, 2, 3, -4] 4 2
5688 [-15, 16, 11, -12] 0 2
5689 [-15, 16, 11, -12] 0 2
5690 [-13, 14, 13, -14] 0 2
5691 [-3, 2, 1, -2] 4 2
5692 [-15, 16, 1, -2] 2 2
5693 [-15, 16, 1, -2] 2 2
5694 [-15, 16, 3, -4] 2 2
5695 [-1, 2, 15, -16] 2 2
5696 [-3, 4, 3, -4] 4 2
5697 [-3, 4, 1, -2] 4 2
5698 [-3, 4, 1, -2] 4 2
5699 [-3, 4, 11, -12] 2 2
5700 [-3, 4, 3, -4] 4 2
5701 [-1, 2, 13, -14] 2 2
5702 [-15, 16, 3, -4] 2 2
5703 [-15, 16, 1, -2] 2 2
5704 [-3, 4, 3, -4] 4 2
5705 [-3, 4, 1, -2] 4 2
5706 [-3, 4, 1

7459 [-1, 2, 15, -16] 2 2
7460 [-15, 16, 3, -4] 2 2
7461 [-15, 16, 15, -16] 0 2
7462 [-1, 2, 3, -4] 4 2
7463 [-1, 2, 3, -4] 4 2
7464 [-11, 12, 1, -2] 2 2
7465 [-15, 16, 15, -16] 0 2
7466 [-1, 2, 3, -4] 4 2
7467 [-11, 12, 1, -2] 2 2
7468 [-1, 2, 1, -2] 4 2
7469 [-1, 2, 15, -16] 2 2
7470 [-1, 2, 13, -14] 2 2
7471 [-1, 2, 13, -14] 2 2
7472 [-1, 2, 13, -14] 2 2
7473 [-1, 2, 1, -2] 4 2
7474 [-11, 12, 3, -4] 2 2
7475 [-1, 2, 3, -4] 4 2
7476 [-1, 2, 3, -4] 4 2
7477 [-15, 16, 1, -2] 2 2
7478 [-1, 2, 3, -4] 4 2
7479 [-1, 2, 3, -4] 4 2
7480 [-13, 14, 1, -2] 2 2
7481 [-1, 2, 1, -2] 4 2
7482 [-1, 2, 1, -2] 4 2
7483 [-1, 2, 3, -4] 4 2
7484 [-13, 14, 1, -2] 2 2
7485 [-15, 16, 3, -4] 2 2
7486 [-3, 4, 3, -4] 4 2
7487 [-1, 2, 1, -2] 4 2
7488 [-13, 14, 1, -2] 2 2
7489 [-15, 16, 1, -2] 2 2
7490 [-1, 2, 3, -4] 4 2
7491 [-3, 4, 3, -4] 4 2
7492 [-3, 4, 13, -14] 2 2
7493 [-3, 4, 3, -4] 4 2
7494 [-3, 4, 1, -2] 4 2
7495 [-15, 16, 1, -2] 2 2
7496 [-13, 14, 3, -4] 2 2
7497 [-3, 4, 13, -14] 2 2
7498 [-15, 16, 3, 

10443 [-15, 16, 3, -4] 2 2
10444 [-11, 12, 3, -4] 2 2
10445 [-1, 2, 13, -14] 2 2
10446 [-1, 2, 3, -4] 4 2
10447 [-13, 14, 1, -2] 2 2
10448 [-3, 4, 13, -14] 2 2
10449 [-15, 16, 15, -16] 0 2
10450 [-1, 2, 15, -16] 2 2
10451 [-1, 2, 1, -2] 4 2
10452 [-3, 4, 13, -14] 2 2
10453 [-15, 16, 1, -2] 2 2
10454 [-3, 4, 1, -2] 4 2
10455 [-1, 2, 11, -12] 2 2
10456 [-15, 16, 3, -4] 2 2
10457 [-15, 16, 1, -2] 2 2
10458 [-13, 14, 1, -2] 2 2
10459 [-1, 2, 3, -2] 4 2
10460 [-11, 12, 1, -2] 2 2
10461 [-1, 2, 3, -4] 4 2
10462 [-1, 2, 3, -4] 4 2
10463 [-1, 2, 3, -4] 4 2
10464 [-1, 2, 1, -2] 4 2
10465 [-15, 16, 3, -4] 2 2
10466 [-15, 16, 3, -4] 2 2
10467 [-1, 4, 1, -2] 4 2
10468 [-3, 4, 13, -14] 2 2
10469 [-3, 2, 15, -16] 2 2
10470 [-3, 4, 11, -12] 2 2
10471 [-3, 4, 3, -4] 4 2
10472 [-1, 2, 1, -2] 4 2
10473 [-15, 16, 3, -4] 2 2
10474 [-15, 16, 3, -4] 2 2
10475 [-3, 4, 13, -14] 2 2
10476 [-1, 2, 1, -2] 4 2
10477 [-11, 12, 13, -14] 0 2
10478 [-15, 16, 3, -4] 2 2
10479 [-11, 12, 1, -2] 2 2
10480 [-3, 4, 3, -4] 

11598 [-11, 12, 11, -12] 0 2
11599 [-1, 2, 15, -16] 2 2
11600 [-15, 16, 3, -4] 2 2
11601 [-3, 4, 1, -2] 4 2
11602 [-3, 4, 1, -2] 4 2
11603 [-15, 16, 1, -2] 2 2
11604 [-1, 2, 3, -4] 4 2
11605 [-3, 4, 3, -4] 4 2
11606 [-1, 2, 1, -2] 4 2
11607 [-3, 4, 1, -2] 4 2
11608 [-1, 2, 3, -4] 4 2
11609 [-1, 2, 1, -2] 4 2
11610 [-15, 16, 1, -2] 2 2
11611 [-1, 2, 3, -4] 4 2
11612 [-3, 4, 13, -14] 2 2
11613 [-13, 14, 1, -2] 2 2
11614 [-1, 2, 3, -4] 4 2
11615 [-11, 12, 1, -2] 2 2
11616 [-3, 4, 3, -4] 4 2
11617 [-1, 2, 1, -2] 4 2
11618 [-1, 2, 3, -4] 4 2
11619 [-15, 16, 15, -16] 0 2
11620 [-1, 2, 1, -2] 4 2
11621 [-15, 16, 15, -16] 0 2
11622 [-13, 14, 13, -14] 0 2
11623 [-1, 2, 3, -4] 4 2
11624 [-11, 12, 1, -2] 2 2
11625 [-1, 2, 3, -4] 4 2
11626 [-3, 4, 3, -2] 4 2
11627 [-13, 14, 1, -2] 2 2
11628 [-11, 12, 3, -4] 2 2
11629 [-11, 12, 3, -4] 2 2
11630 [-3, 4, 13, -14] 2 2
11631 [-15, 16, 3, -4] 2 2
11632 [-1, 2, 11, -12] 2 2
11633 [-3, 4, 11, -12] 2 2
11634 [-3, 4, 1, -2] 4 2
11635 [-1, 2, 3, -4] 4 2
1163

14475 [-3, 4, 3, -4] 4 2
14476 [-1, 2, 1, -2] 4 2
14477 [-1, 2, 1, -2] 4 2
14478 [-11, 12, 11, -12] 0 2
14479 [-1, 2, 13, -14] 2 2
14480 [-1, 2, 3, -4] 4 2
14481 [-3, 4, 15, -16] 2 2
14482 [-1, 2, 13, -14] 2 2
14483 [-3, 4, 1, -2] 4 2
14484 [-3, 4, 1, -4] 4 2
14485 [-1, 2, 3, -4] 4 2
14486 [-15, 16, 3, -4] 2 2
14487 [-3, 4, 13, -14] 2 2
14488 [-5, 4, 3, -4] 4 2
14489 [-11, 12, 3, -4] 2 2
14490 [-13, 14, 3, -4] 2 2
14491 [-1, 2, 3, -4] 4 2
14492 [-1, 2, 3, -4] 4 2
14493 [-13, 14, 3, -4] 2 2
14494 [-15, 16, 15, -16] 0 2
14495 [-3, 4, 1, -2] 4 2
14496 [-3, 4, 3, -4] 4 2
14497 [-3, 4, 1, -2] 4 2
14498 [-11, 12, 13, -14] 0 2
14499 [-1, 2, 1, -2] 4 2
14500 [-1, 2, 3, -4] 4 2
14501 [-15, 16, 1, -2] 2 2
14502 [-15, 16, 3, -4] 2 2
14503 [-1, 2, 11, -12] 2 2
14504 [-11, 12, 1, -2] 2 2
14505 [-15, 16, 1, -2] 2 2
14506 [-15, 16, 15, -16] 0 2
14507 [-3, 4, 3, -4] 4 2
14508 [-1, 2, 1, -2] 4 2
14509 [-11, 12, 3, -4] 2 2
14510 [-13, 14, 11, -12] 0 2
14511 [-1, 2, 15, -16] 2 2
14512 [-13, 14, 11, -12] 

15555 [-3, 4, 3, -4] 4 2
15556 [-3, 4, 15, -16] 2 2
15557 [-3, 4, 3, -4] 4 2
15558 [-3, 4, 1, -2] 4 2
15559 [-15, 16, 15, -16] 0 2
15560 [-1, 2, 1, -2] 4 2
15561 [-1, 2, 3, -4] 4 2
15562 [-15, 16, 1, -2] 2 2
15563 [-13, 14, 1, -2] 2 2
15564 [-3, 4, 3, -4] 4 2
15565 [-1, 2, 15, -16] 2 2
15566 [-13, 14, 13, -14] 0 2
15567 [-15, 16, 1, -2] 2 2
15568 [-1, 2, 3, -4] 4 2
15569 [-1, 2, 1, -2] 4 2
15570 [-3, 4, 15, -16] 2 2
15571 [-3, 4, 1, -2] 4 2
15572 [-13, 14, 1, -4] 2 2
15573 [-13, 14, 15, -16] 0 2
15574 [-3, 4, 13, -14] 2 2
15575 [-11, 12, 11, -12] 0 2
15576 [-15, 16, 11, -12] 0 2
15577 [-15, 16, 13, -14] 0 2
15578 [-1, 2, 1, -2] 4 2
15579 [-1, 2, 1, -4] 4 2
15580 [-13, 14, 3, -4] 2 2
15581 [-3, 4, 1, -2] 4 2
15582 [-1, 2, 1, -2] 4 2
15583 [-3, 4, 13, -14] 2 2
15584 [-3, 4, 15, -16] 2 2
15585 [-1, 2, 13, -14] 2 2
15586 [-3, 4, 3, -4] 4 2
15587 [-1, 2, 1, -2] 4 2
15588 [-13, 14, 3, -4] 2 2
15589 [-3, 2, 13, -14] 2 2
15590 [-3, 4, 3, -4] 4 2
15591 [-3, 4, 3, -4] 4 2
15592 [-3, 4, 1, -2] 4 

In [19]:
ak.any(photon_mask)

NameError: name 'photon_mask' is not defined

In [ ]:
8352+2564+2616+2593+1875

In [ ]:
ak.sum(genHVVVars["fj_H_VV_elenuqq"])+ak.sum(genHVVVars["fj_H_VV_munuqq"])+ak.sum(genHVVVars["fj_H_VV_4q"])+ak.sum(genHVVVars["fj_H_VV_leptauelvqq"])+ak.sum(genHVVVars["fj_H_VV_leptaumuvqq"])+ak.sum(genHVVVars["fj_H_VV_hadtauvqq"])+ak.sum((num_leptons==2))

In [ ]:
ak.sum(genHVVVars["fj_H_VV_munuqq"])

In [ ]:
ak.sum((num_taus == 1) & (num_quarks == 2) & (num_leptons==1))

In [ ]:
ak.sum((num_taus == 1) & (taudecay == 3) & (num_quarks == 2) & (num_leptons==1))

In [ ]:
ak.sum((num_taus == 1) & (num_quarks == 2) & (num_leptons==1) & (taudecay==0))

In [ ]:
ak.sum(genHVVVars["fj_H_VV_leptauelvqq"])+ak.sum(genHVVVars["fj_H_VV_leptaumuvqq"])+ak.sum(genHVVVars["fj_H_VV_hadtauvqq"])

In [ ]:
ak.sum((num_leptons==2))

In [ ]:
ak.sum(higgs.pt>0)

In [ ]:
ak.sum(genHVVVars["fj_H_VV_isMatched"])

In [ ]:
for i,m in enumerate(lep_daughters[fatjet.delta_r(lep_daughters) < 0.8].distinctParent.mass):
    print(i,m,v.mass[i],v_star.mass[i],iswstarlepton[i])

In [ ]:
genHVVVars["fj_nquarks"]

In [ ]:
daughters_pdgId = ak.where(photon_mask, ak.flatten(abs(daughters.children.pdgId),axis=-1), abs(daughters.pdgId))
flat_daughters_pdgId = ak.flatten(ak.flatten(daughters_pdgId,axis=3),axis=1)

In [ ]:
daughters.children.children.pdgId[148]

In [ ]:
all_daughters = ak.where(photon_mask, ak.flatten(daughters.children,axis=4), daughters)
all_daughters = ak.flatten(ak.flatten(all_daughters,axis=3),axis=2)
all_daughters.pdgId[148]

In [ ]:
ak.flatten(ak.flatten(abs(daughters.pdgId),axis=2),axis=2)[55]


In [ ]:
photon_mask = ak.any(flat_daughters_pdgId == 22,axis=2)
photon_mask[55]

In [ ]:
daughters[0].pdgId


In [ ]:
ak.flatten(daughters.children,axis=4)[55].pdgId

In [ ]:
all_daughters.pdgId[1]

In [ ]:
ak.where(photon_mask, daughters.children, daughters).pdgId[55]

In [ ]:
photon_mask

In [ ]:
matched_higgs.distinctChildren.pdgId[55]

In [ ]:
#child = matched_higgs.distinctChildren.children.pdgId
child = matched_higgs.children.children.pdgId
child = higgs.children.children.pdgId

for i,d in enumerate(child):
    #print(i,d,num_quarks[i],num_leptons[i],num_taus[i],((num_quarks!=4) & (num_leptons==0))[i],flat_daughters_pdgId[i])

In [ ]:
print(matched_higgs_children.children.children.pdgId[55])

In [ ]:
dchild = matched_higgs_children.distinctChildren.pdgId
print(dchild[55])

In [ ]:
a = ak.flatten(ak.flatten(matched_higgs_children.children.pdgId,axis=1))
a
b = ak.flatten(ak.flatten(matched_higgs_children.distinctChildren.pdgId,axis=1))
b


In [ ]:
ak.flatten(ak.flatten(matched_higgs_children.distinctChildren.pdgId[matched_higgs_children.children.pdgId==24],axis=1))

In [ ]:
print("higgs pt ", matched_higgs.pt[decay != 6])
print("w mass ", v.mass[decay != 6])
print("children ", matched_higgs_children.pdgId[decay[selection] != 6])
print("children ", matched_higgs_children.children.pdgId[decay != 6])
print("distinct children ", matched_higgs_children.distinctChildren.pdgId[decay != 6])
print("decay ", decay[decay != 6])
print("dau id ", daughters_pdgId[decay != 6])


In [ ]:
b[a==24]

In [ ]:
munuqq = (num_muons == 1) & (num_quarks == 2) & (num_leptons==1)
#for i,l in enumerate(parent[munuqq].mass):
#    print(i,l,iswlepton[munuqq][i],iswstarlepton[munuqq][i])
print(ak.sum(munuqq))
print(ak.sum(iswlepton[munuqq]))
print(ak.sum(iswstarlepton[munuqq]))

#print(parent[munuqq].mass.to_numpy())
#print(iswlepton[munuqq].to_numpy())
#print(iswstarlepton[munuqq])

In [ ]:
lep_daughters = all_daus_flat[leptons]
parent = ak.firsts(lep_daughters.distinctParent)
parent_mass = parent.mass
iswlepton = parent.mass == v.mass
iswstarlepton = parent.mass == v_star.mass
print(ak.sum(munuqq))
print(ak.sum(iswlepton[munuqq]))
print(ak.sum(iswstarlepton[munuqq]))
print(1251+1212)
#print(iswlepton[munuqq].to_numpy())

In [ ]:
print(lep_daughters.pt)
print(lep_daughters.pdgId)
print(lep_daughters.distinctParent.mass)
print(ak.firsts(lep_daughters.distinctParent).mass)

print(lep_daughters[munuqq].pdgId)
print(lep_daughters[munuqq].distinctParent.mass)
print(ak.firsts(lep_daughters.distinctParent)[munuqq].mass)